# System Recommendation - Artikel (Word2Vec - Cosine Similarity)

In [16]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

#### Import Artikel

In [8]:
df = pd.read_csv("../../data/artikel-kesehatan-mental/artikel.csv")

In [9]:
df.head()

,Judul_Artikel,Unnamed: 1,Unnamed: 2,Link_Artikel,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Isi_Artikel
0,Mental Sehat di Hari Senin Ini 5 Triknya,NaN,NaN,https://www.halodoc.com/artikel/mental-sehat-d...,NaN,NaN,NaN,NaN,NaN,"Bagi sebagian besar orang, memulai kembali har..."
1,Ketahui 6 Cara Mendukung Kesehatan Mental Anak,NaN,NaN,https://www.halodoc.com/artikel/ketahui-6-cara...,NaN,NaN,NaN,NaN,NaN,Setiap orang tua pasti ingin yang terbaik untu...
2,Manfaat Membaca Buku bagi Kesehatan Mental,NaN,NaN,https://www.halodoc.com/artikel/manfaat-membac...,NaN,NaN,NaN,NaN,NaN,Membaca buku menjadi salah satu kegiatan yang ...
3,Kesehatan Mental Orang Tua Berpengaruh pada Anak,NaN,NaN,https://www.halodoc.com/artikel/kesehatan-ment...,NaN,NaN,NaN,NaN,NaN,Kesehatan mental orang tua berhubungan dengan ...
4,"Sering Diabaikan, Ini Pentingnya Menjaga Keseh...",NaN,NaN,https://www.halodoc.com/artikel/sering-diabaik...,NaN,NaN,NaN,NaN,NaN,Sangat mudah bagi orangtua untuk mengidentifik...


In [10]:
df = df.dropna(axis=1)

In [11]:
df.head()

,Judul_Artikel,Link_Artikel,Isi_Artikel
0,Mental Sehat di Hari Senin Ini 5 Triknya,https://www.halodoc.com/artikel/mental-sehat-d...,"Bagi sebagian besar orang, memulai kembali har..."
1,Ketahui 6 Cara Mendukung Kesehatan Mental Anak,https://www.halodoc.com/artikel/ketahui-6-cara...,Setiap orang tua pasti ingin yang terbaik untu...
2,Manfaat Membaca Buku bagi Kesehatan Mental,https://www.halodoc.com/artikel/manfaat-membac...,Membaca buku menjadi salah satu kegiatan yang ...
3,Kesehatan Mental Orang Tua Berpengaruh pada Anak,https://www.halodoc.com/artikel/kesehatan-ment...,Kesehatan mental orang tua berhubungan dengan ...
4,"Sering Diabaikan, Ini Pentingnya Menjaga Keseh...",https://www.halodoc.com/artikel/sering-diabaik...,Sangat mudah bagi orangtua untuk mengidentifik...


#### Preprocessing

In [17]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("indonesian"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [18]:
df['cleaned'] = df['Isi_Artikel'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(func=make_lower_case)
df['cleaned'] = df.cleaned.apply(func=remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)

In [19]:
df.head()

,Judul_Artikel,Link_Artikel,Isi_Artikel,cleaned
0,Mental Sehat di Hari Senin Ini 5 Triknya,https://www.halodoc.com/artikel/mental-sehat-d...,"Bagi sebagian besar orang, memulai kembali har...",orang senin rutinitas menyeramkan heran minggu...
1,Ketahui 6 Cara Mendukung Kesehatan Mental Anak,https://www.halodoc.com/artikel/ketahui-6-cara...,Setiap orang tua pasti ingin yang terbaik untu...,orang tua terbaik kesehatan anaknya segi fisik...
2,Manfaat Membaca Buku bagi Kesehatan Mental,https://www.halodoc.com/artikel/manfaat-membac...,Membaca buku menjadi salah satu kegiatan yang ...,membaca buku salah kegiatan saja anak anak ora...
3,Kesehatan Mental Orang Tua Berpengaruh pada Anak,https://www.halodoc.com/artikel/kesehatan-ment...,Kesehatan mental orang tua berhubungan dengan ...,kesehatan mental orang tua berhubungan kondisi...
4,"Sering Diabaikan, Ini Pentingnya Menjaga Keseh...",https://www.halodoc.com/artikel/sering-diabaik...,Sangat mudah bagi orangtua untuk mengidentifik...,mudah orangtua mengidentifikasi kebutuhan fisi...


#### Load Pretrained Word2Vec Indonesia

In [26]:
word2vec_model = Word2Vec.load("../../pretrained-model/idwiki_word2vec_300.model")

#### Compute TF-IDF Value

In [28]:
#Building TFIDF model and calculate TFIDF score
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5)
tfidf.fit(df['cleaned'])

# Getting the words from the TF-IDF model

tfidf_list = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
tfidf_feature = tfidf.get_feature_names() # tfidf words/col-names

In [30]:
# Building TF-IDF Word2Vec 

# Storing the TFIDF Word2Vec embeddings
tfidf_vectors = []; 
line = 0;
# for each book description
for desc in corpus: 
  # Word vectors are of zero length (Used 300 dimensions)
    sent_vec = np.zeros(300) 
    # num of words with a valid vector in the book description
    weight_sum =0; 
    # for each word in the book description
    for word in desc: 
        if word in word2vec_model.wv.key_to_index and word in tfidf_feature:
            vec = word2vec_model.wv[word]
            tf_idf = tfidf_list[word] * (desc.count(word) / len(desc))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_vectors.append(sent_vec)
    line += 1

#### Recommendation

In [36]:
#Recommending top 5 similar books
def recommendations(title):
    # finding cosine similarity for the vectors
    cosine_similarities = cosine_similarity(tfidf_vectors,  tfidf_vectors)
    
    # taking the title and book image link and store in new data frame called books
    books = df[['Judul_Artikel', 'Link_Artikel']]
    #Reverse mapping of the index
    indices = pd.Series(df.index, index = df['Judul_Artikel']).drop_duplicates()
         
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    recommend = books.iloc[book_indices]
    for index, row in recommend.iterrows():
        print(row['Judul_Artikel'])

#### Result

In [35]:
recommendations("Mental Sehat di Hari Senin Ini 5 Triknya")

Cara Menjaga Kesehatan Mental di Lingkungan Kerja
Pentingnya Menjaga Kesehatan Mental Pekerja Selama WFH
Jangan Diabaikan Kesehatan Mental Ibu Pasca Melahirkan
Tips Ampuh Menjaga Kesehatan Mental Selama PPKM
Nike Liburkan Seminggu Karyawannya Demi Jaga Kesehatan Mental


#### Save Similarity For Each Article To CSV

In [45]:
similarity_df = pd.DataFrame(cosine_similarity(tfidf_vectors,  tfidf_vectors))

In [46]:
similarity_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.000000,0.637097,0.577162,0.633479,0.488326,0.724715,0.798578,0.551694,0.700128,0.874739,...,0.716984,0.484924,0.714885,0.751754,0.840517,0.733768,0.871642,0.447905,0.636351,0.657957
1,0.637097,1.000000,0.476673,0.945289,0.946470,0.571723,0.635069,0.496162,0.561221,0.756990,...,0.642455,0.935454,0.672328,0.666350,0.712135,0.614897,0.726779,0.386311,0.551073,0.691664
2,0.577162,0.476673,1.000000,0.443159,0.398897,0.447156,0.438616,0.350988,0.445906,0.525513,...,0.450884,0.382264,0.446308,0.451718,0.506887,0.402729,0.518883,0.390777,0.472272,0.384033
3,0.633479,0.945289,0.443159,1.000000,0.872419,0.570115,0.667502,0.506270,0.555413,0.779976,...,0.633512,0.866802,0.671736,0.675785,0.760399,0.668439,0.751063,0.396880,0.554727,0.709852
4,0.488326,0.946470,0.398897,0.872419,1.000000,0.379474,0.442532,0.322631,0.368504,0.592806,...,0.455420,0.995171,0.468805,0.463354,0.566537,0.492532,0.550270,0.259829,0.392363,0.471723


In [85]:
similarity_df.stack().to_frame().to_csv(path_or_buf='../../data/formatted_data/artikel_similarity.csv')